We want to obtain the Hugging face model, that is built in call_api, giving it the same parameters as the simplest of calls (single niah for mistral). Then, I create a new jsonl file containing the prompt, the answer, and both. Tokenization is non deterministic so test on multiple samples.

In [4]:
from model_wrappers import HuggingFaceModel
model_name_or_path="lmsys/longchat-7b-v1.5-32k"
temperature=0.0
top_k=1
top_p=1.0
stop_words=""
tokens_to_generate=128
#We load the LLM
llm=HuggingFaceModel(
    name_or_path=model_name_or_path,
    do_sample=temperature > 0,
    repetition_penalty=1,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    stop=stop_words,
    max_new_tokens=tokens_to_generate,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[NeMo W 2024-07-30 08:41:44 nemo_logging:349] /mloscratch/homes/easselin/conda/envs/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
      warnings.warn(
    
[NeMo W 2024-07-30 08:41:44 nemo_logging:349] /mloscratch/homes/easselin/conda/envs/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corres

For debugging and comprehension purposes, this cell is to run call_api.py:

In [6]:
%%sh
MODEL_NAME="lmsys/longchat-7b-v1.5-32k"
ROOT_DIR="../../results"  
MODEL_DIR="../../models"
BENCHMARK="synthetic"
MAX_SEQ_LENGTH="16384"
RESULTS_DIR="${ROOT_DIR}/${MODEL_NAME}/${BENCHMARK}/${MAX_SEQ_LENGTH}"
DATA_DIR="${RESULTS_DIR}/data"
PRED_DIR="${RESULTS_DIR}/pred"
TASK="niah_single_1"
MODEL_FRAMEWORK="hf"
MODEL_PATH=$MODEL_NAME
TEMPERATURE="0.0"
TOP_P="1.0"
TOP_K="1"
 python call_api.py \
            --data_dir ${DATA_DIR} \
            --save_dir ${PRED_DIR} \
            --benchmark ${BENCHMARK} \
            --task ${TASK} \
            --server_type ${MODEL_FRAMEWORK} \
            --model_name_or_path ${MODEL_PATH} \
            --temperature ${TEMPERATURE} \
            --top_k ${TOP_K} \
            --top_p ${TOP_P} \


Process is interrupted.


[NeMo W 2024-07-26 14:20:33 nemo_logging:349] /mloscratch/homes/easselin/conda/envs/env/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


Predict niah_single_1 
from ../../results/lmsys/longchat-7b-v1.5-32k/synthetic/16384/data/niah_single_1/validation.jsonl
to ../../results/lmsys/longchat-7b-v1.5-32k/synthetic/16384/pred/niah_single_1.jsonl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s][NeMo W 2024-07-26 14:20:47 nemo_logging:349] /mloscratch/homes/easselin/conda/envs/env/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
      return self.fget.__get__(instance, owner)()
    
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.45s/it]
[NeMo W 2024-07-26 14:20:54 nemo_logging:349] /mloscratch/homes/easselin/conda/envs/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected

# Tokenization


In [5]:
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest
import torch
from tqdm import tqdm
task='niah_single_1'
contextlength=4096
strcontextlength=str(contextlength)
example_path="examples/"+model_name_or_path+"/synthetic/"+strcontextlength+"/example_"+task+".jsonl"
data=read_manifest(example_path)

In [6]:
for i in tqdm(range(len(data))):
    variables=data[i]
    prompt=variables["prompt"]
    answer=variables["answer"]
    concat=variables["concatenation"]
    # assert prompt+answer==concat
    prompttokenization=llm.tokenizer(prompt,return_tensors="pt").input_ids
    answertokenization=llm.tokenizer(answer,return_tensors="pt",add_special_tokens=False).input_ids
    concattokenization=llm.tokenizer(concat,return_tensors="pt").input_ids
    # print(f"Prompt:       {' '.join(map(str, prompttokenization.tolist()))}")
    # print(f"Answer:       {' '.join(map(str, answertokenization.tolist()))}")
    # print(f"Concatenation:{' '.join(map(str, concattokenization.tolist()))}")
    # print(len(answertokenization[0].tolist()))
    assert torch.allclose(torch.cat([prompttokenization,answertokenization],1),concattokenization)
    # break

100%|██████████| 500/500 [00:05<00:00, 83.65it/s]


### If we don't remove the special tokens on the answer

We can observe that if we remove the first two tokens of the answer's tokenization, then the concatenation of the tokenization
of the prompt and the tokenization of the answer is the same as the tokenization of the concatenation of the prompt and the answer.

### If we remove the special tokens on the answer

#### If we put no spaces 

```
-End of prompt:       "...,2245, 28804]"
-Beginning of answer: "[                 415, 2841, 9693,..."
-Transition in concat:"...,2245, 28804, 1014, 2841, 9693,..."
```

We can observe that here the first token of the answer differs between the two cases.

#### If we put spaces

```
-End of prompt:       "...,2245, 28804]"
-Beginning of answer: "[                28705, 415, 2841, 9693,..."
-Transition in concat:"...,2245, 28804,        415, 2841, 9693,..."
```

We can observe that here there is an additional token at the beginning of the answer, even though we deactivated the special tokens (including BOS).

#### If we put double spaces

```
-End of prompt:       "...,2245, 28804]"
-Beginning of answer: "[                  259, 415, 2841, 9693,..."
-Transition in concat:"...,2245, 28804, 28705, 415, 2841, 9693,..."
```

There is again a difference in the first token of the answer in the two cases.

# Perplexity
Here we implement perplexity evaluation for different configurations.

In [7]:
losses=[]
model=llm.pipeline.model if llm.pipeline else llm.model
tokenizer=llm.pipeline.tokenizer if llm.pipeline else llm.tokenizer
device=model.device
stride=250
maxtokens=contextlength-stride
initnkeeplast=stride
for nkeeplast in range(initnkeeplast,maxtokens,stride):
    for sample in tqdm(data):

        prompt=sample["prompt"]
        answer=sample["answer"]
        
        
        
        prompttokens=tokenizer(prompt,return_tensors="pt").to(device).input_ids
        answertokens=tokenizer(answer,return_tensors="pt",add_special_tokens=False).to(device).input_ids
        concattokens=torch.cat([prompttokens,answertokens],1)
        lengthconcat=len(concattokens[0].tolist())
        
        tokens = concattokens[:,-nkeeplast:]
        labels = tokens.clone()
        
        
        
        lengthofanswer=len(answertokens[0].tolist())
        labels[:,:-lengthofanswer]=-100

        

        with torch.no_grad():
            outputs=model(tokens, labels=labels)
        
        loss = outputs.loss
        losses.append(loss)
        
    ppl=torch.exp(torch.stack(losses).mean())
    print(f"Average perplexity over the answer by passing the last {nkeeplast} tokens of the concatenation of the prompt and the answer as context:{ppl}")
    # print(f"Average perplexity:{ppl}")
    losses.clear()

100%|██████████| 500/500 [00:21<00:00, 23.72it/s]


Average perplexity over the answer by passing the last 250 tokens of the concatenation of the prompt and the answer as context:3.520951747894287


100%|██████████| 500/500 [00:23<00:00, 21.41it/s]


Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:3.2366271018981934


100%|██████████| 500/500 [00:30<00:00, 16.32it/s]


...

Average perplexity over the answer by passing the last 3250 tokens of the concatenation of the prompt and the answer as context:1.6058282852172852


100%|██████████| 500/500 [02:19<00:00,  3.60it/s]


Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:1.4817001819610596


100%|██████████| 500/500 [02:23<00:00,  3.47it/s]


Average perplexity over the answer by passing the last 3750 tokens of the concatenation of the prompt and the answer as context:1.403790831565857


## Needle in the haystack task 1
### Mistral 7B
#### Max context length of 4096

Average perplexity over the answer by passing only the answer as context:174.9510040283203  
Average perplexity over the answer by passing the last 250 tokens of the concatenation of the prompt and the answer as context:3.442931890487671  
Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:3.0682711601257324  
Average perplexity over the answer by passing the last 750 tokens of the concatenation of the prompt and the answer as context:2.8815793991088867  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:2.6554274559020996  
Average perplexity over the answer by passing the last 1250 tokens of the concatenation of the prompt and the answer as context:2.4824306964874268  
Average perplexity over the answer by passing the last 1500 tokens of the concatenation of the prompt and the answer as context:2.3085646629333496  
Average perplexity over the answer by passing the last 1750 tokens of the concatenation of the prompt and the answer as context:2.1747803688049316  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.0302517414093018  
Average perplexity over the answer by passing the last 2250 tokens of the concatenation of the prompt and the answer as context:1.8966240882873535  
Average perplexity over the answer by passing the last 2500 tokens of the concatenation of the prompt and the answer as context:1.7730286121368408  
Average perplexity over the answer by passing the last 2750 tokens of the concatenation of the prompt and the answer as context:1.6829378604888916  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:1.5539112091064453  
Average perplexity over the answer by passing the last 3250 tokens of the concatenation of the prompt and the answer as context:1.445020079612732  
Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:1.3325660228729248  
Average perplexity over the answer by passing the last 3750 tokens of the concatenation of the prompt and the answer as context:1.2605961561203003  
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.4215729236602783  

#### Max context length of 8192

Average perplexity over the answer by passing only the answer as context:175.12060546875  
Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:3.25126051902771  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:2.9953126907348633  
Average perplexity over the answer by passing the last 1500 tokens of the concatenation of the prompt and the answer as context:2.8163704872131348  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.563474655151367  
Average perplexity over the answer by passing the last 2500 tokens of the concatenation of the prompt and the answer as context:2.3883209228515625  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:2.251896858215332  
Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:2.1382880210876465  
Average perplexity over the answer by passing the last 4000 tokens of the concatenation of the prompt and the answer as context:2.0110976696014404  
Average perplexity over the answer by passing the last 4500 tokens of the concatenation of the prompt and the answer as context:1.8779411315917969  
Average perplexity over the answer by passing the last 5000 tokens of the concatenation of the prompt and the answer as context:1.7482768297195435  
Average perplexity over the answer by passing the last 5500 tokens of the concatenation of the prompt and the answer as context:1.6497514247894287  
Average perplexity over the answer by passing the last 6000 tokens of the concatenation of the prompt and the answer as context:1.525417447090149  
Average perplexity over the answer by passing the last 6500 tokens of the concatenation of the prompt and the answer as context:1.4379204511642456  
Average perplexity over the answer by passing the last 7000 tokens of the concatenation of the prompt and the answer as context:1.3026238679885864  
Average perplexity over the answer by passing the last 7500 tokens of the concatenation of the prompt and the answer as context:1.2399232387542725  
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.3250515460968018  

Less perplexity for 8192 might be due to the fact that the model was trained to have context length 32k tokens?

#### Max context length of 16384

Average perplexity over the answer by passing only the answer as context:176.14071655273438  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:3.2502217292785645  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.955977439880371  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:2.786891460418701  
Average perplexity over the answer by passing the last 4000 tokens of the concatenation of the prompt and the answer as context:2.5911900997161865  
Average perplexity over the answer by passing the last 5000 tokens of the concatenation of the prompt and the answer as context:2.435096502304077  
Average perplexity over the answer by passing the last 6000 tokens of the concatenation of the prompt and the answer as context:2.2886531352996826  
Average perplexity over the answer by passing the last 7000 tokens of the concatenation of the prompt and the answer as context:2.172229766845703  
Average perplexity over the answer by passing the last 8000 tokens of the concatenation of the prompt and the answer as context:2.0517468452453613  
Average perplexity over the answer by passing the last 9000 tokens of the concatenation of the prompt and the answer as context:1.886850357055664  
Average perplexity over the answer by passing the last 10000 tokens of the concatenation of the prompt and the answer as context:1.7527867555618286  
Average perplexity over the answer by passing the last 11000 tokens of the concatenation of the prompt and the answer as context:1.6625641584396362  
Average perplexity over the answer by passing the last 12000 tokens of the concatenation of the prompt and the answer as context:1.5579078197479248  
Average perplexity over the answer by passing the last 13000 tokens of the concatenation of the prompt and the answer as context:1.45879065990448  
Average perplexity over the answer by passing the last 14000 tokens of the concatenation of the prompt and the answer as context:1.3523118495941162  
Average perplexity over the answer by passing the last 15000 tokens of the concatenation of the prompt and the answer as context:1.2737163305282593  
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.307148814201355  

### LWM

#### Max context length of 4096

Average perplexity over the answer by passing only the answer as context:78.059326171875  
Average perplexity over the answer by passing the last 250 tokens of the concatenation of the prompt and the answer as context:2.7902402877807617  
Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:2.6105449199676514  
Average perplexity over the answer by passing the last 750 tokens of the concatenation of the prompt and the answer as context:2.483823776245117  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:2.3147709369659424  
Average perplexity over the answer by passing the last 1250 tokens of the concatenation of the prompt and the answer as context:2.172853708267212  
Average perplexity over the answer by passing the last 1500 tokens of the concatenation of the prompt and the answer as context:2.0439436435699463  
Average perplexity over the answer by passing the last 1750 tokens of the concatenation of the prompt and the answer as context:1.943989634513855  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:1.8406829833984375  
Average perplexity over the answer by passing the last 2250 tokens of the concatenation of the prompt and the answer as context:1.7273703813552856  
Average perplexity over the answer by passing the last 2500 tokens of the concatenation of the prompt and the answer as context:1.6065398454666138  
Average perplexity over the answer by passing the last 2750 tokens of the concatenation of the prompt and the answer as context:1.5346611738204956  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:1.4086531400680542  
Average perplexity over the answer by passing the last 3250 tokens of the concatenation of the prompt and the answer as context:1.32203209400177  
Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:1.2222048044204712  
Average perplexity over the answer by passing the last 3750 tokens of the concatenation of the prompt and the answer as context:1.1569379568099976  
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.1488076448440552  


#### Max context length of 8192

Average perplexity over the answer by passing only the answer as context:78.05083465576172  
Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:2.7583587169647217  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:2.5906922817230225  
Average perplexity over the answer by passing the last 1500 tokens of the concatenation of the prompt and the answer as context:2.4765517711639404  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.304326057434082  
Average perplexity over the answer by passing the last 2500 tokens of the concatenation of the prompt and the answer as context:2.1398379802703857  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:2.0182175636291504  
Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:1.9385747909545898  
Average perplexity over the answer by passing the last 4000 tokens of the concatenation of the prompt and the answer as context:1.8298826217651367  
Average perplexity over the answer by passing the last 4500 tokens of the concatenation of the prompt and the answer as context:1.7096558809280396  
Average perplexity over the answer by passing the last 5000 tokens of the concatenation of the prompt and the answer as context:1.6005035638809204  
Average perplexity over the answer by passing the last 5500 tokens of the concatenation of the prompt and the answer as context:1.5128042697906494  
Average perplexity over the answer by passing the last 6000 tokens of the concatenation of the prompt and the answer as context:1.3970189094543457  
Average perplexity over the answer by passing the last 6500 tokens of the concatenation of the prompt and the answer as context:1.3171504735946655  
Average perplexity over the answer by passing the last 7000 tokens of the concatenation of the prompt and the answer as context:1.2068501710891724  
Average perplexity over the answer by passing the last 7500 tokens of the concatenation of the prompt and the answer as context:1.151483416557312    
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.1486250162124634  

#### Max context length of 16384

Average perplexity over the answer by passing only the answer as context:78.03204345703125  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:2.793048858642578  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.635413408279419  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:2.4776053428649902  
Average perplexity over the answer by passing the last 4000 tokens of the concatenation of the prompt and the answer as context:2.3439559936523438  
Average perplexity over the answer by passing the last 5000 tokens of the concatenation of the prompt and the answer as context:2.202911853790283  
Average perplexity over the answer by passing the last 6000 tokens of the concatenation of the prompt and the answer as context:2.0741262435913086  
Average perplexity over the answer by passing the last 7000 tokens of the concatenation of the prompt and the answer as context:1.981422781944275  
Average perplexity over the answer by passing the last 8000 tokens of the concatenation of the prompt and the answer as context:1.884514570236206  
Average perplexity over the answer by passing the last 9000 tokens of the concatenation of the prompt and the answer as context:1.746207594871521  
Average perplexity over the answer by passing the last 10000 tokens of the concatenation of the prompt and the answer as context:1.6279515027999878    
Average perplexity over the answer by passing the last 11000 tokens of the concatenation of the prompt and the answer as context:1.5399411916732788  
Average perplexity over the answer by passing the last 12000 tokens of the concatenation of the prompt and the answer as context:1.4471083879470825  
Average perplexity over the answer by passing the last 13000 tokens of the concatenation of the prompt and the answer as context:1.3545198440551758  
Average perplexity over the answer by passing the last 14000 tokens of the concatenation of the prompt and the answer as context:1.2605664730072021   
Average perplexity over the answer by passing the last 15000 tokens of the concatenation of the prompt and the answer as context:1.1903263330459595   
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the asnwer as context:1.1593486070632935

### LongChat

#### Max context length of 4096

Average perplexity over the answer by passing only the answer as context:84.53561401367188  
Average perplexity over the answer by passing the last 250 tokens of the concatenation of the prompt and the answer as context:3.520951747894287  
Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:3.2366271018981934  
Average perplexity over the answer by passing the last 750 tokens of the concatenation of the prompt and the answer as context:3.0646657943725586  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:2.8605432510375977  
Average perplexity over the answer by passing the last 1250 tokens of the concatenation of the prompt and the answer as context:2.676612377166748  
Average perplexity over the answer by passing the last 1500 tokens of the concatenation of the prompt and the answer as context:2.4917852878570557  
Average perplexity over the answer by passing the last 1750 tokens of the concatenation of the prompt and the answer as context:2.354177236557007  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.2453253269195557  
Average perplexity over the answer by passing the last 2250 tokens of the concatenation of the prompt and the answer as context:2.0984466075897217  
Average perplexity over the answer by passing the last 2500 tokens of the concatenation of the prompt and the answer as context:1.9636598825454712  
Average perplexity over the answer by passing the last 2750 tokens of the concatenation of the prompt and the answer as context:1.8619531393051147  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:1.7078495025634766  
Average perplexity over the answer by passing the last 3250 tokens of the concatenation of the prompt and the answer as context:1.6058282852172852  
Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:1.4817001819610596  
Average perplexity over the answer by passing the last 3750 tokens of the concatenation of the prompt and the answer as context:1.403790831565857  
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.626822590827942  

#### Max context length of 8192

Average perplexity over the answer by passing only the answer as context:84.63575744628906  
Average perplexity over the answer by passing the last 500 tokens of the concatenation of the prompt and the answer as context:3.4246301651000977  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:3.199336051940918   
Average perplexity over the answer by passing the last 1500 tokens of the concatenation of the prompt and the answer as context:3.0150558948516846  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:2.8123795986175537  
Average perplexity over the answer by passing the last 2500 tokens of the concatenation of the prompt and the answer as context:2.599802017211914  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:2.4217867851257324  
Average perplexity over the answer by passing the last 3500 tokens of the concatenation of the prompt and the answer as context:2.3292667865753174  
Average perplexity over the answer by passing the last 4000 tokens of the concatenation of the prompt and the answer as context:2.175149917602539  
Average perplexity over the answer by passing the last 4500 tokens of the concatenation of the prompt and the answer as context:2.036252975463867  
Average perplexity over the answer by passing the last 5000 tokens of the concatenation of the prompt and the answer as context:1.9171148538589478  
Average perplexity over the answer by passing the last 5500 tokens of the concatenation of the prompt and the answer as context:1.8126466274261475  
Average perplexity over the answer by passing the last 6000 tokens of the concatenation of the prompt and the answer as context:1.6782187223434448  
Average perplexity over the answer by passing the last 6500 tokens of the concatenation of the prompt and the answer as context:1.5899064540863037  
Average perplexity over the answer by passing the last 7000 tokens of the concatenation of the prompt and the answer as context:1.4453948736190796  
Average perplexity over the answer by passing the last 7500 tokens of the concatenation of the prompt and the answer as context:1.3939419984817505  
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.6423969268798828

#### Max context length of 16384

Average perplexity over the answer by passing only the answer as context:84.92120361328125  
Average perplexity over the answer by passing the last 1000 tokens of the concatenation of the prompt and the answer as context:3.449686050415039  
Average perplexity over the answer by passing the last 2000 tokens of the concatenation of the prompt and the answer as context:3.2106401920318604  
Average perplexity over the answer by passing the last 3000 tokens of the concatenation of the prompt and the answer as context:2.953179359436035  
Average perplexity over the answer by passing the last 4000 tokens of the concatenation of the prompt and the answer as context:2.772296667098999  
Average perplexity over the answer by passing the last 5000 tokens of the concatenation of the prompt and the answer as context:2.618698835372925  
Average perplexity over the answer by passing the last 6000 tokens of the concatenation of the prompt and the answer as context:2.453761577606201  
Average perplexity over the answer by passing the last 7000 tokens of the concatenation of the prompt and the answer as context:2.3301615715026855  
Average perplexity over the answer by passing the last 8000 tokens of the concatenation of the prompt and the answer as context:2.231750011444092  
Average perplexity over the answer by passing the last 9000 tokens of the concatenation of the prompt and the answer as context:2.0827157497406006  
Average perplexity over the answer by passing the last 10000 tokens of the concatenation of the prompt and the answer as context:1.9356590509414673  
Average perplexity over the answer by passing the last 11000 tokens of the concatenation of the prompt and the answer as context:1.8202787637710571  
Average perplexity over the answer by passing the last 12000 tokens of the concatenation of the prompt and the answer as context:1.7264333963394165  
Average perplexity over the answer by passing the last 13000 tokens of the concatenation of the prompt and the answer as context:1.6375830173492432  
Average perplexity over the answer by passing the last 14000 tokens of the concatenation of the prompt and the answer as context:1.5031508207321167  
Average perplexity over the answer by passing the last 15000 tokens of the concatenation of the prompt and the answer as context:1.4270881414413452   
Average perplexity over the answer by passing all the tokens of the concatenation of the prompt and the answer as context:1.5213795900344849 






